In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)

* Importing data

In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import data
import pandas as pd

train_data = pd.read_csv('/content/drive/MyDrive/Тестовое задание на стажировку ML-инженер/trainee_train.csv', index_col=0)
test_data = pd.read_csv('/content/drive/MyDrive/Тестовое задание на стажировку ML-инженер/trainee_test_fish.csv', index_col=0)

* Exploration analysis

In [ ]:
train_data.head()

,im,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v437,v438,v439,v440,v441,v442,v443,v444,v445,v446
0,0.0,128.100243,128.157072,129.255102,128.273006,126.532819,129.023232,129.854839,129.828431,129.868526,...,128.800985,128.232694,128.211893,130.512476,130.411160,130.650165,130.904685,131.017268,130.477398,130.496150
1,1.0,128.100243,128.157072,126.636364,128.273006,130.810403,129.023232,128.633333,128.994152,128.994152,...,128.914365,128.232694,128.211893,128.167519,130.411160,130.650165,130.904685,131.017268,130.477398,130.496150
2,1.0,128.100243,128.157072,129.255102,128.273006,130.810403,129.023232,126.769231,126.926295,127.196460,...,128.800985,128.232694,128.211893,128.167519,128.051724,127.709786,127.349282,127.063052,125.169118,125.274194
3,1.0,128.100243,128.157072,127.630027,128.273006,126.532819,129.023232,126.769231,126.926295,127.196460,...,128.800985,130.794007,131.026119,130.512476,130.411160,130.650165,130.904685,131.017268,130.477398,130.496150
4,1.0,128.100243,128.336364,129.255102,128.273006,130.810403,126.042980,129.854839,126.926295,127.196460,...,128.800985,130.794007,131.026119,130.512476,130.411160,130.650165,130.904685,131.017268,130.477398,130.496150


In [ ]:
test_data.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v437,v438,v439,v440,v441,v442,v443,v444,v445,v446
0,130.798507,131.174905,129.255102,132.199234,128.746479,130.381148,129.854839,129.828431,129.868526,130.917969,...,128.800985,128.232694,128.211893,128.167519,126.100000,127.709786,127.349282,127.063052,127.825000,126.611554
1,128.100243,128.157072,127.630027,128.273006,130.810403,129.023232,126.769231,126.926295,127.196460,127.433333,...,128.800985,130.794007,127.750000,130.512476,130.411160,130.650165,130.904685,131.017268,130.477398,130.496150
2,128.100243,128.157072,129.255102,128.273006,130.810403,126.042980,129.854839,129.828431,129.868526,128.618063,...,128.800985,130.794007,131.026119,130.512476,130.411160,130.650165,130.904685,131.017268,130.477398,130.496150
3,130.798507,131.174905,129.255102,132.199234,132.860465,130.381148,129.854839,129.828431,129.868526,130.917969,...,126.503759,128.232694,128.211893,128.167519,128.051724,127.709786,127.349282,127.063052,127.825000,126.611554
4,130.798507,131.174905,129.255102,132.199234,130.810403,130.381148,129.854839,129.828431,129.868526,130.917969,...,128.914365,128.232694,128.211893,128.167519,128.051724,127.709786,127.349282,127.063052,125.169118,126.611554


In [ ]:
# Check missing values
sum(train_data.isna().sum())

0

In [ ]:
# Splitting into train and test data + IQR outliers detection
x_train = train_data.drop('im', axis=1)
x_test = test_data

for column in x_train.columns:
    Q1 = x_train[column].quantile(0.25)
    Q3 = x_train[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR

    # Calculate mean or median without outliers in the training data
    mean_value = x_train[(x_train[column] >= lower_bound) & (x_train[column] <= upper_bound)][column].mean()

    # Replace outliers in both training and test data
    x_train.loc[(x_train[column] < lower_bound) | (x_train[column] > upper_bound), column] = mean_value
    x_test.loc[(x_test[column] < lower_bound) | (x_test[column] > upper_bound), column] = mean_value

X_train = x_train.to_numpy()
Y_train = train_data['im'].to_numpy()

x_test = x_test.to_numpy()

In [ ]:
# Class imbalance noticed, lets fix this problem with SMOTE, which we will solve within pipeline during cross_validation
from collections import Counter
Counter(Y_train)

Counter({0.0: 174, 1.0: 442})

In [ ]:
!pip install imbalanced-learn

* Standartizing + KFold (It's better to observe mean score on every possible split)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_val_score

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=52)

* Baseline (Logistic Regression Model)




In [ ]:
from sklearn.linear_model import LogisticRegression
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.metrics import roc_auc_score

pipe = Pipeline(steps=[
    ('smote', BorderlineSMOTE(random_state=52, kind='borderline-1')),
    ('standardscaler', StandardScaler()),
    ('logisticregression', LogisticRegression(random_state=52))
])

roc_auc_scores = cross_val_score(pipe, X_train, Y_train, cv=kfold, scoring='roc_auc')
print(roc_auc_scores)
print(roc_auc_scores.mean())

[0.94574639 0.89415584 0.89220779 0.91266234 0.89160608]
0.9072756886206985


* Approach: SVM

In [ ]:
from sklearn.svm import SVC

pipe = Pipeline(steps=[
    ('smote', BorderlineSMOTE(random_state=52, kind='borderline-1')),
    ('standardscaler', StandardScaler()),
    ('svmmodel', SVC(kernel='rbf', C=1.0, random_state=52))
])

roc_auc_scores = cross_val_score(pipe, X_train, Y_train, cv=kfold, scoring='roc_auc')
print(roc_auc_scores)
print(roc_auc_scores.mean())

[0.96083467 0.93603896 0.92694805 0.91201299 0.87541309]
0.9222495515059956


In [ ]:
!pip install bayesian-optimization

In [ ]:
# Trying to pick the best parameters with PCA

import numpy as np
from bayes_opt import BayesianOptimization
from sklearn.decomposition import PCA


def svm_bo(C, kernel='rbf'):

  params = {
      'svm__C': C,
      'svm__kernel': kernel,
      # 'pca__n_components': int(n_components)
  }

  pipe = Pipeline(steps=[
      ('smote', BorderlineSMOTE(random_state=52, kind='borderline-1')),
      ('standardscaler', StandardScaler()),
      # ('pca', PCA()),
      ('svm', SVC(random_state=52))
  ])

  pipe.set_params(**params)
  scores = cross_val_score(pipe, X_train, Y_train, cv=kfold, scoring='roc_auc')
  return scores.mean()



params_svm = {
    'C': (1, 20),
    # 'n_components': (200, 440)
}

# Initialize Bayesian Optimization
svm_bo = BayesianOptimization(svm_bo, params_svm, random_state=52)
svm_bo.maximize(init_points=5, n_iter=20)

best = svm_bo.max['params']
print(best)

|   iter    |  target   |     C     |
-------------------------------------
| 1         | 0.9251    | 16.64     |
| 2         | 0.9214    | 1.496     |
| 3         | 0.9249    | 5.005     |
| 4         | 0.9251    | 12.75     |
| 5         | 0.9241    | 2.867     |
| 6         | 0.9251    | 18.14     |
| 7         | 0.9251    | 14.05     |
| 8         | 0.9251    | 20.0      |
| 9         | 0.9251    | 10.63     |
| 10        | 0.9251    | 8.546     |
| 11        | 0.9251    | 6.87      |
| 12        | 0.9251    | 15.35     |
| 13        | 0.9251    | 9.6       |
| 14        | 0.9251    | 11.71     |
| 15        | 0.9251    | 19.11     |
| 16        | 0.9244    | 3.982     |
| 17        | 0.9248    | 5.972     |
| 18        | 0.9251    | 7.707     |
| 19        | 0.9251    | 17.37     |
| 20        | 0.9251    | 14.7      |
| 21        | 0.9251    | 13.37     |
| 22        | 0.9251    | 16.0      |
| 23        | 0.9251    | 9.073     |
| 24        | 0.9251    | 11.17     |
| 25        

* Approach: CatBoostClassifier

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 5.0 MB/s eta 0:00:00


In [ ]:
# Try to find the best parameters

# Optimal parameters with PCA and SS: {'depth': 6.796939520449791, 'iterations': 853.199215524583, 'learning_rate': 0.2914911435217489}
# Target    | depth     | iter      | lr
# 0.9262    | 6.797     | 853.2     | 0.2915

# In theory PCA + SS + SMOTE will not boost our solution with CatBoost,
# because CatBoost search through every feature and return the best splitting value
# Features which are not important won't affect on convergence


# TODO: Try to remove PCA and SS in pipeline
import numpy as np
from bayes_opt import BayesianOptimization
from sklearn.decomposition import PCA
from catboost import CatBoostClassifier


def catboost_bo(learning_rate, iterations, depth):

  params = {
      'catboost__learning_rate':  learning_rate,
      'catboost__iterations': int(iterations),
      'catboost__depth': int(depth),
      'catboost__loss_function': 'Logloss',
      'catboost__custom_metric': 'AUC',
  }

  pipe = Pipeline(steps=[
      # ('smote', BorderlineSMOTE(random_state=52, kind='borderline-1')),
      # ('standardscaler', StandardScaler()),
      ('catboost', CatBoostClassifier(logging_level='Silent'))
  ])

  pipe.set_params(**params)
  scores = cross_val_score(pipe, X_train, Y_train, cv=kfold, scoring='roc_auc')
  return scores.mean()



params_catboost = {
    'learning_rate':  (0.1, 1),
    'iterations': (100, 1000),
    'depth': (2, 9),
}

# Initialize Bayesian Optimization
catboost_bo = BayesianOptimization(catboost_bo, params_catboost, random_state=111)
catboost_bo.maximize(init_points=5, n_iter=25)

best = catboost_bo.max['params']
print(best)

|   iter    |  target   |   depth   | iterat... | learni... |
-------------------------------------------------------------
| 1         | 0.9184    | 6.285     | 252.2     | 0.4925    |
| 2         | 0.9237    | 7.385     | 365.8     | 0.2342    |
| 3         | 0.9152    | 2.157     | 478.2     | 0.3148    |
| 4         | 0.9216    | 4.364     | 991.6     | 0.314     |
| 5         | 0.9127    | 2.568     | 702.6     | 0.6591    |
| 6         | 0.9268    | 6.728     | 366.7     | 0.2943    |
| 7         | 0.9149    | 5.157     | 368.3     | 0.3317    |
| 8         | 0.9121    | 8.036     | 367.1     | 0.4215    |
| 9         | 0.9082    | 6.572     | 366.6     | 0.6263    |
| 10        | 0.9226    | 7.683     | 645.7     | 0.2834    |
| 11        | 0.9174    | 2.707     | 677.3     | 0.1611    |
| 12        | 0.9238    | 8.888     | 925.2     | 0.114     |
| 13        | 0.9127    | 8.604     | 959.8     | 0.4705    |
| 14        | 0.9126    | 3.078     | 911.8     | 0.6067    |
| 15    

* Approach: The best parameters of XGBoost

In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBClassifier

# Optimal parameters with PCA and SS:
# |  target   | colsam... |   gamma   | learni... | max_depth | n_esti... | subsample |
# | 0.9317    | 0.6379    | 1.476     | 0.1127    | 14.73     | 703.8     | 0.7479    |
def xgb_bo(max_depth, learning_rate, n_estimators, gamma, subsample, colsample_bytree):

    params = {
        'xgbclassifier__max_depth': int(max_depth),
        'xgbclassifier__learning_rate': learning_rate,
        'xgbclassifier__n_estimators': int(n_estimators),
        'xgbclassifier__gamma': gamma,
        'xgbclassifier__subsample': subsample,
        'xgbclassifier__colsample_bytree': colsample_bytree,
        'xgbclassifier__use_label_encoder': False,
        'xgbclassifier__eval_metric': 'logloss'
    }

    pipe = Pipeline(steps=[
      ('smote', BorderlineSMOTE(random_state=52, kind='borderline-1')),
      ('standardscaler', StandardScaler()),
      ('xgbclassifier', XGBClassifier(random_state=52))
    ])


    #Train the model
    pipe.set_params(**params)

    scores = cross_val_score(pipe, X_train, Y_train, cv=kfold, scoring='roc_auc')

    return scores.mean()

#Hyperparameter space
params_xgb = {
    'max_depth': (3, 20),
    'learning_rate': (0.01, 0.5),
    'n_estimators': (100, 2000),
    'gamma': (0, 10),
    'subsample': (0.5, 1),
    'colsample_bytree': (0.5, 1),
}

# Initialize Bayesian Optimization
xgb_bo = BayesianOptimization(xgb_bo, params_xgb, random_state=52)
xgb_bo.maximize(init_points=10, n_iter=50)

best = xgb_bo.max['params']
print(best)

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------------------------------
| 1         | 0.9181    | 0.8061    | 1.691     | 0.2237    | 16.08     | 661.1     | 0.5746    |
| 2         | 0.9256    | 0.5112    | 4.202     | 0.127     | 8.74      | 1.982e+03 | 0.6189    |
| 3         | 0.9123    | 0.5406    | 6.696     | 0.3144    | 7.662     | 985.8     | 0.5592    |
| 4         | 0.9027    | 0.537     | 9.008     | 0.399     | 17.29     | 1.649e+03 | 0.9955    |
| 5         | 0.9152    | 0.7886    | 8.138     | 0.2164    | 3.467     | 962.9     | 0.5527    |
| 6         | 0.9158    | 0.9086    | 6.977     | 0.287     | 7.662     | 1.997e+03 | 0.569     |
| 7         | 0.9203    | 0.8077    | 4.85      | 0.2085    | 15.37     | 713.3     | 0.7003    |
| 8         | 0.9002    | 0.6595    | 9.472     | 0.4602    | 16.84     | 164.7     | 0.9717    |
| 9         | 0.9169

In [ ]:
# Another model with additional last 3 parameters

# |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | reg_alpha | reg_la... | subsample |
# | 0.9325    | 0.868     | 0.9236    | 0.3456    | 5.327     | 2.809     | 532.5     | 0.331     | 0.1515    | 0.9338   |
from xgboost import XGBClassifier

def xgb_bo(max_depth, learning_rate, n_estimators, gamma, subsample, colsample_bytree, min_child_weight, reg_lambda, reg_alpha):
    params = {
        'xgbclassifier__max_depth': int(max_depth),
        'xgbclassifier__learning_rate': learning_rate,
        'xgbclassifier__n_estimators': int(n_estimators),
        'xgbclassifier__gamma': gamma,
        'xgbclassifier__subsample': subsample,
        'xgbclassifier__colsample_bytree': colsample_bytree,
        'xgbclassifier__use_label_encoder': False,
        'xgbclassifier__eval_metric': 'logloss',
        'xgbclassifier__min_child_weight': min_child_weight,
        'xgbclassifier__reg_lambda': reg_lambda,
        'xgbclassifier__reg_alpha': reg_alpha
    }

    pipe = Pipeline(steps=[
      ('smote', BorderlineSMOTE(random_state=52, kind='borderline-1')),
      ('standardscaler', StandardScaler()),
      ('xgbclassifier', XGBClassifier(random_state=52))
    ])


    #Train the model
    pipe.set_params(**params)

    scores = cross_val_score(pipe, X_train, Y_train, cv=kfold, scoring='roc_auc')

    return scores.mean()

#Hyperparameter space
params_xgb = {
    'max_depth': (3, 20),
    'learning_rate': (0.01, 0.5),
    'n_estimators': (100, 2000),
    'gamma': (0, 10),
    'subsample': (0.5, 1),
    'colsample_bytree': (0.5, 1),
    'min_child_weight': (1, 10),
    'reg_lambda': (0, 8),
    'reg_alpha': (0, 8)
}

# Initialize Bayesian Optimization
xgb_bo = BayesianOptimization(xgb_bo, params_xgb, random_state=52)
xgb_bo.maximize(init_points=10, n_iter=100)

best = xgb_bo.max['params']
print(best)

* Approach: Random Forest

In [ ]:
# Doesn't make sense
from sklearn.ensemble import RandomForestClassifier


def rf_bo(max_depth, n_estimators, min_samples_split, min_samples_leaf, max_leaf_nodes, max_features):

    params = {
        'rf__max_depth': int(max_depth),
        'rf__n_estimators': int(n_estimators),
        'rf__min_samples_split': int(min_samples_split),
        'rf__min_samples_leaf': int(min_samples_leaf),
        'rf__max_leaf_nodes': int(max_leaf_nodes),
        'rf__max_features': max_features
    }

    pipe = Pipeline(steps=[
      ('smote', BorderlineSMOTE(random_state=42, kind='borderline-1')),
      ('standardscaler', StandardScaler()),
      ('rf', RandomForestClassifier())
    ])

    #Train the model
    pipe.set_params(**params)
    scores = cross_val_score(pipe, X_train, Y_train, cv=kfold, scoring='roc_auc')

    return scores.mean()

#Hyperparameter space
params_rf = {
    'max_depth': (2, 50),
    'n_estimators': (100, 1000),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 10),
    'max_leaf_nodes': (10, 1000),
    'max_features': (0.1, 1)
}

# Initialize Bayesian Optimization
xgb_bo = BayesianOptimization(rf_bo, params_rf, random_state=111)
xgb_bo.maximize(init_points=10, n_iter=50)

best = rf_bo.max['params']
print(best)

|   iter    |  target   | max_depth | max_fe... | max_le... | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------------------
| 1         | 0.9219    | 31.38     | 0.2522    | 441.7     | 7.923     | 4.363     | 234.2     |
| 2         | 0.8957    | 3.079     | 0.4782    | 246.3     | 4.039     | 9.926     | 314.0     |
| 3         | 0.9149    | 5.897     | 0.7026    | 625.0     | 3.468     | 5.73      | 206.5     |
| 4         | 0.9142    | 5.55      | 0.9107    | 796.0     | 8.565     | 8.522     | 991.9     |
| 5         | 0.9226    | 29.71     | 0.8324    | 427.1     | 1.247     | 5.633     | 194.8     |
| 6         | 0.9236    | 41.23     | 0.728     | 569.6     | 3.468     | 9.988     | 224.2     |
| 7         | 0.9187    | 31.54     | 0.5365    | 411.0     | 7.551     | 4.583     | 460.5     |
| 8         | 0.9173    | 17.31     | 0.9525    | 919.5     | 8.325     | 2.273     | 949.0     |
| 9         | 0.9167

KeyboardInterrupt: 

* Making final prediction on test set !!! ✊

In [ ]:
from imblearn.over_sampling import BorderlineSMOTE
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler

params = {
    'colsample_bytree': 0.868,
    'gamma': 0.9236,
    'learning_rate': 0.3456,
    'max_depth': int(5.327),
    'min_child_weight': 2.809,
    'n_estimators': int(532.5),
    'reg_lambda': 0.331,
    'reg_alpha': 0.1515,
    'subsample': 0.9338,
    'use_label_encoder': False,
    'eval_metric': 'logloss',
}

# Defining models
smote = BorderlineSMOTE(kind='borderline-1')
scaler = StandardScaler()
model = XGBClassifier(**params)

# Prepare data
X_train_smote, Y_train_smote = smote.fit_resample(X_train, Y_train)
X_train_scaled = scaler.fit_transform(X_train_smote)

# Train the model
model.fit(X_train_scaled, Y_train_smote)

# Making prediction
X_test_scaled = scaler.transform(x_test)
predictions = model.predict(X_test_scaled)

In [ ]:
df = pd.DataFrame({
    'im': predictions
})

df.to_csv('/content/sample_data/answers.csv')
df.head()

,im
0,0
1,1
2,0
3,0
4,0


In [ ]:
# Testing

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


params = {
    'colsample_bytree': 0.868,
    'gamma': 0.9236,
    'learning_rate': 0.3456,
    'max_depth': int(5.327),
    'min_child_weight': 2.809,
    'n_estimators': int(532.5),
    'reg_lambda': 0.331,
    'reg_alpha': 0.1515,
    'subsample': 0.9338,
    'use_label_encoder': False,
    'eval_metric': 'logloss',
}

# Defining models
smote = BorderlineSMOTE(kind='borderline-1')
scaler = StandardScaler()
model = XGBClassifier(**params)

x_train_train, x_train_val, y_train_train, y_train_val = train_test_split(X_train, Y_train, test_size=0.1, shuffle=True)

X_train_smote, Y_train_smote = smote.fit_resample(x_train_train, y_train_train)
X_train_scaled = scaler.fit_transform(X_train_smote)
X_val_scaled = scaler.transform(x_train_val)

# Train the model
model.fit(X_train_scaled, Y_train_smote)

# Making prediction
predictions = model.predict_proba(X_val_scaled)
roc_auc_score(y_train_val, predictions[:, 1])

0.9777777777777776